<a href="https://www.kaggle.com/code/noelsudhish/mnist-digit-recog?scriptVersionId=99688102" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Activation, Dense, BatchNormalization, Dropout

In [3]:
digit_train=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
X_test=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
dig_sub=pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [4]:
digit_train.shape

(42000, 785)

In [5]:
 X_test.isnull().sum()

pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 784, dtype: int64

In [6]:
X_test.shape

(28000, 784)

In [7]:
#y_train=to_categorical(digit_train.pop("label"))
y_train=digit_train.pop("label")
X_train=digit_train


# Scaling input Training and Test Data

In [8]:
X_train=X_train/255.0
X_test=X_test/255.0

# One hot encoding the output values(categories 0-9) in Dense format, so sparse is set to False

In [9]:
len(y_train.values)

42000

In [10]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
y_train=enc.fit_transform(y_train.values.reshape(-1,1))


In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(42000, 784)
(28000, 784)
(42000, 10)


# **2 hidden layers with dropout Batch normalisation and Early stopping**

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(70, input_shape = (784,), activation = 'relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(70, activation = 'relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(70, activation = 'relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax',kernel_initializer='he_normal'))

2022-06-30 10:13:07.289853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 10:13:07.452491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 10:13:07.453473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-30 10:13:07.455518: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 70)                54950     
_________________________________________________________________
batch_normalization (BatchNo (None, 70)                280       
_________________________________________________________________
dropout (Dropout)            (None, 70)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 70)                4970      
_________________________________________________________________
batch_normalization_1 (Batch (None, 70)                280       
_________________________________________________________________
dropout_1 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 70)                4

### Early Stopping which checks if loss value is decreasing for 30 epochs

In [14]:
keras_callbacks   = [EarlyStopping(monitor='val_loss', patience=30, min_delta=0,restore_best_weights=True)]


## **Learning Rate is 0.001**

In [15]:
adam = optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
model.fit(X_train, y_train, batch_size = 56, epochs = 100,validation_split = 0.2, verbose = 1)

2022-06-30 10:13:11.466892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
600/600 [==============================] - 5s 5ms/step - loss: 0.6488 - accuracy: 0.7998 - val_loss: 0.2262 - val_accuracy: 0.9299
Epoch 2/100
600/600 [==============================] - 3s 5ms/step - loss: 0.3162 - accuracy: 0.9059 - val_loss: 0.1804 - val_accuracy: 0.9432
Epoch 3/100
600/600 [==============================] - 3s 4ms/step - loss: 0.2569 - accuracy: 0.9226 - val_loss: 0.1536 - val_accuracy: 0.9521
Epoch 4/100
600/600 [==============================] - 3s 4ms/step - loss: 0.2210 - accuracy: 0.9345 - val_loss: 0.1378 - val_accuracy: 0.9585
Epoch 5/100
600/600 [==============================] - 3s 4ms/step - loss: 0.1980 - accuracy: 0.9400 - val_loss: 0.1227 - val_accuracy: 0.9613
Epoch 6/100
600/600 [==============================] - 3s 5ms/step - loss: 0.1744 - accuracy: 0.9474 - val_loss: 0.1196 - val_accuracy: 0.9640
Epoch 7/100
600/600 [==============================] - 3s 4ms/step - loss: 0.1635 - accuracy: 0.9515 - val_loss: 0.1110 - val_accuracy: 0.9658

In [17]:
train_results = model.evaluate(X_train,y_train)

1313/1313 [==============================] - 5s 4ms/step - loss: 0.0248 - accuracy: 0.9940


In [18]:
print(model.metrics_names)    
print("\nTraining : ",train_results)

['loss', 'accuracy']

Training :  [0.024816248565912247, 0.9940237998962402]


In [19]:
y_pred = model.predict(X_test)


## Output is in one hot encoded format,so we need to bring it back to the original format(reverse one hot encoding)
## 10 column(OHE) to 1 column(Original)

In [20]:
np.floor(np.exp(y_pred)) #10 columns output

array([[1., 1., 2., ..., 1., 1., 1.],
       [2., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 2.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 2.],
       [1., 1., 2., ..., 1., 1., 1.]], dtype=float32)

In [21]:
out = pd.DataFrame(np.floor(np.exp(y_pred)))

### Reversing onehot encoding using simple idxmax of columns

In [22]:
out['Label']=out.idxmax(1)   


In [23]:
out['ImageId']=X_test.index+1


In [24]:
out.drop(out.iloc[:,0:10],axis=1,inplace=True)

### Rearanging column order so Predicted label is last column

In [25]:
cols=out.columns.tolist()
cols=cols[-1:] + cols[:-1]
out=out[cols]


In [26]:
out.to_csv('submission.csv',index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [27]:
pd.read_csv("submission.csv")

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
